# Process VIIRS Night-time lights data

The data we use are the monthly DNB (day-night band) composites.

Download these from 
https://www.ngdc.noaa.gov/eog/viirs/download_dnb_composites.html
or https://eogdata.mines.edu/download_dnb_composites.html

There are 6 tiles for each global monthly image; they are at 500m resolution. They are provided in .tgz files and they are very large due to the resolution and the nature of the data meaning they do not compress well. 

This notebook helps organise the downloaded files, unzip, and merge them.

In [1]:
import os

In [2]:
from osgeo import gdal

In [3]:
import zipfile
import tarfile

In [4]:
import subprocess

In [5]:
import glob
import shutil

In [6]:
from joblib import Parallel, delayed


ModuleNotFoundError: No module named 'joblib'

In [28]:
indir = r'E:\Data\Harry\Documents\dataprep\VIIRS_annual'


Move all the files from a single download folder into subfolders corresponding to the date

In [31]:
infiles = glob.glob(indir+'\\*.tgz')

In [32]:
for f in infiles:
    base = os.path.basename(f)
    daterange = base.split('_')[2]
    destdir = os.path.join(indir,daterange)
    if not os.path.exists(destdir):
        os.mkdir(destdir)
    shutil.move(f, destdir)
    

In [33]:
indirs = [d for d in glob.glob(indir+"\\*") if os.path.isdir(d)]


In [34]:
sorted(indirs)

['E:\\Data\\Harry\\Documents\\dataprep\\VIIRS_annual\\20150101-20151231',
 'E:\\Data\\Harry\\Documents\\dataprep\\VIIRS_annual\\20160101-20161231']

Define a function which mosaics, translates, and prepares tiles into a single output tiff. We just do this by shelling out to the gdal command line tools.

In [112]:
vrtcommand = "gdalbuildvrt {0} {1}"
transcommand = "gdal_translate -of GTiff -co COMPRESS=LZW "+\
    "-co PREDICTOR=2 -co TILED=YES -co SPARSE_OK=TRUE -co BIGTIFF=YES "+\
    "--config GDAL_CACHEMAX 8000 {0} {1}"
ovcommand = "gdaladdo -ro --config COMPRESS_OVERVIEW LZW --config USE_RRD NO " +\
        "--config TILED YES {0} 2 4 8 16 32 64 128 256 --config GDAL_CACHEMAX 8000"
statcommand = "gdalinfo -stats {0} >nul"
def translateDir(dIn, dOut=None, viirsSDR="SVDNB", filecode="vcmcfg", ext="avg_rade9"):
    if dOut is None:
        dOut = dIn
    print ("Operating in folder "+dIn)
    filetag = "SVDNB" #'vcmslcfg' # or vcmcfg for stray-light-excluded
    tars = glob.glob(dIn+"\\"+"*"+filetag+"*.tgz")
    if len(tars)<6:
        print ("incomplete folder!")
        return
    dirTileList = []
    subDirName = os.path.basename(dIn)
    outFile = os.path.join(dOut, "VIIRS.DNB.StrayLightCorrected."+subDirName)
    
    if os.path.exists(outFile + ".tif"):
        print ("Already done! Skipping")
        return
    #print outFile + " doesn't exist: running!"
    #return
    for t in tars:
        # the tar file will have 7 parts same as the tiffs it contains for the monthly ones, but 
        # only 6 parts for the annual ones which contain multiple shortnames
        if t.find(filetag) == -1:
            continue
        tar = tarfile.open(t)
        #tardir = os.path.dirname(t)
        members = tar.getmembers()
        for m in members:
            fnFull = os.path.basename(m.name)
            try:
                fn, theirExt, tifExt = fnFull.split('.')
                viirssdr, sat, daterange, roi, shortname, version, date = fn.split('_')
            except ValueError:
                print("unparseable filename {} in tar {}, skipping".format(fnFull, t))
                continue
            if ((theirExt.find(ext) != -1)
                and (shortname == filecode)):
                dirTileList.append(os.path.join(dIn, fnFull))
                if os.path.exists(os.path.join(dIn, fnFull)):
                    print (fnFull +" exists - skipping extraction!")
                    continue
                print ("Extracting " + fnFull)
                
                tar.extract(m, dIn)
                
            else:
                print("skipped " +fnFull)
    
    vrtBuilder = vrtcommand.format(outFile+".vrt", " ".join(dirTileList))
    #vrtBuilder = vrtcommand.format(outFile+".vrt", os.path.join(dIn,"*"+filetag+"*.tif"))
    print (vrtBuilder)
    subprocess.call(vrtBuilder)
        
    transBuilder = transcommand.format(outFile+".vrt", outFile+".tif")
    print (transBuilder)
    subprocess.call(transBuilder)
    
    ovBuilder = ovcommand.format(outFile+".tif")
    print (ovBuilder)
    subprocess.call(ovBuilder)
    
    statBuilder = statcommand.format(outFile+".tif")
    print (statBuilder)
    subprocess.call(statBuilder)
    
    #for f in dirTileList:
    #    os.remove(f)

now just run it on each subfolder, or some of them, or whatever. Tried using the Parallel library to run multiple at once but it didn't work, or something, I can't remember

NB the shortnames are:

In the monthly products:
* vcmcfg
* vcmslcfg

In the annual products:
* vcm-orm
* vcm-orm-ntl
* vcm-ntl

In [75]:
#Parallel(n_jobs=4)(delayed(translateDir)(d) for d in indirs)k
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\Other_Global_Covariates\NightTimeLights\VIIRS_DNB_Monthly'
outDir = r'C:\Temp\nightlights'
for d in sorted(indirs):
    translateDir(d, outDir)

Operating in folder E:\Data\Harry\Documents\dataprep\VIIRS_annual\20150101-20151231
Extracting SVDNB_npp_20150101-20151231_00N060E_vcm-ntl_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060E_vcm-orm_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060W_vcm-ntl_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060W_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060W_vcm-orm_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N060W_vcm_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N180W_vcm-ntl_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N180W_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif
Extracting SVDNB_npp_20150101-20151231_00N

In [113]:
outDir = r'C:\Temp\nightlights'
if not (os.path.isdir(outDir)):
    os.makedirs(outDir)
variablename = "vcm-ntl"
for d in sorted(indirs):
    translateDir(d, outDir, filecode=variablename)


Operating in folder E:\Data\Harry\Documents\dataprep\VIIRS_annual\20150101-20151231
unparseable filename README_dnb_composites_v1.txt in tar E:\Data\Harry\Documents\dataprep\VIIRS_annual\20150101-20151231\SVDNB_npp_20150101-20151231_00N060E_v10_c201701311200.tgz, skipping
SVDNB_npp_20150101-20151231_00N060E_vcm-ntl_v10_c201701311200.avg_rade9.tif exists - skipping extraction!
skipped SVDNB_npp_20150101-20151231_00N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif
skipped SVDNB_npp_20150101-20151231_00N060E_vcm-orm_v10_c201701311200.avg_rade9.tif
skipped SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.avg_rade9.tif
skipped SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.cf_cvg.tif
skipped SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.cvg.tif
unparseable filename README_dnb_composites_v1.txt in tar E:\Data\Harry\Documents\dataprep\VIIRS_annual\20150101-20151231\SVDNB_npp_20150101-20151231_00N060W_v10_c201701311200.tgz, skipping
SVDNB_npp_20150101-20151231_0

In [105]:
t = tarfile.open(r'E:\Data\Harry\Documents\dataprep\VIIRS_annual\20150101-20151231\SVDNB_npp_20150101-20151231_00N060E_v10_c201701311200.tgz')

In [107]:
ms = t.getmembers()
ms

[<TarInfo 'README_dnb_composites_v1.txt' at 0x66098e0>,
 <TarInfo 'SVDNB_npp_20150101-20151231_00N060E_vcm-ntl_v10_c201701311200.avg_rade9.tif' at 0x66099a8>,
 <TarInfo 'SVDNB_npp_20150101-20151231_00N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif' at 0x6609688>,
 <TarInfo 'SVDNB_npp_20150101-20151231_00N060E_vcm-orm_v10_c201701311200.avg_rade9.tif' at 0x6609a70>,
 <TarInfo 'SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.avg_rade9.tif' at 0x6609b38>,
 <TarInfo 'SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.cf_cvg.tif' at 0x6609c00>,
 <TarInfo 'SVDNB_npp_20150101-20151231_00N060E_vcm_v10_c201701311200.cvg.tif' at 0x6609cc8>]

In [ ]:
fn ms